In [424]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from copy import deepcopy
# from math import factorial, exp

## Graph Pre-Processing

In [425]:
## Graph Pre-Processing
def add_virtual_start_node(G):
    roots = [n for n in G.nodes if G.in_degree(n) == 0]
    if len(roots) > 1:
        entry_node = "attacker_entry_node"
        G.add_node(entry_node)
        for r in roots:
            # Probability=1, weight=1 for “virtual edge”
            G.add_edge(entry_node, r, edge_probabilities=1, weight=1)
    else:
        entry_node = roots[0] if roots else None
    return entry_node, roots

In [426]:
def process_target_node(G):
    """
    Merges multiple sink nodes into a single 'c("12","13","14","16")'
    then converts to DiGraph.
    This mirrors the R approach of calling the merged node c("12","13","14","16").
    """
    import math
    import networkx as nx

    targets = [n for n in G.nodes if G.out_degree(n) == 0]
    if len(targets) > 1:
        merged_label = 'c("12","13","14","16")'
        G.add_node(merged_label)

        # replicate edges p->t as p->merged_label
        for t in targets:
            for p in list(G.predecessors(t)):
                for key, attr_dict in G[p][t].items():
                    if not isinstance(attr_dict, dict):
                        attr_dict = {'weight': attr_dict}
                    prob = attr_dict.get("edge_probabilities", 1.0)
                    wt = attr_dict.get("weight", -math.log(prob) if prob > 0 else float('inf'))
                    G.add_edge(p, merged_label, edge_probabilities=prob, weight=wt)

        # remove old targets
        for t in targets:
            G.remove_node(t)

        # unify multiple edges p->merged_label
        aggregator_dict = {}
        for p in list(G.predecessors(merged_label)):
            for key, attr_dict in G[p][merged_label].items():
                if not isinstance(attr_dict, dict):
                    attr_dict = {'weight': attr_dict}
                p_i = attr_dict.get("edge_probabilities", 1.0)

                if (p, merged_label) not in aggregator_dict:
                    aggregator_dict[(p, merged_label)] = p_i
                else:
                    existing_p = aggregator_dict[(p, merged_label)]
                    # union-of-probabilities aggregator
                    new_p = 1 - (1 - existing_p) * (1 - p_i)
                    aggregator_dict[(p, merged_label)] = new_p

        # remove all multi-edges p->merged_label, then add single aggregated
        for (p, mlabel) in list(aggregator_dict.keys()):
            for k in list(G[p][mlabel].keys()):
                G.remove_edge(p, mlabel, k)
            agg_prob = aggregator_dict[(p, mlabel)]
            if agg_prob <= 0:
                final_wt = float('inf')
            elif agg_prob >= 1:
                final_wt = 0.0
            else:
                final_wt = -math.log(agg_prob)
            G.add_edge(p, mlabel, edge_probabilities=agg_prob, weight=final_wt)

    # convert MultiDiGraph -> DiGraph
    G_converted = nx.DiGraph()
    G_converted.add_nodes_from(G.nodes(data=True))
    for (u, v) in G.edges():
        for _, data in G[u][v].items():
            if not isinstance(data, dict):
                data = {'weight': data}
            if not G_converted.has_edge(u, v):
                G_converted.add_edge(u, v, **data)
    return G_converted


In [427]:
def process_graph(G):
    """
    Merges multiple root nodes into a single virtual entry node (if needed)
    and merges multiple targets into 'virtual_target_node' (if needed).
    Returns a DiGraph, plus the set of original roots and the entry node name.
    """
    import networkx as nx

    entry_node, roots = add_virtual_start_node(G)
    G_final = process_target_node(G)  # returns DiGraph

    # Debug statements right after we've merged the targets
    print("\n[DEBUG] After process_target_node:")
    print(f"   Number of nodes in 'G_final' = {G_final.number_of_nodes()}")
    print("   Node labels:", list(G_final.nodes))

    return processed_graph, original_roots, virtual_entry_node, virtual_target_node


In [428]:
%run "attack_graph_MIR100.ipynb"

## Calculate Game Elements

In [429]:
## Calculate Game Elements
def generate_game_elements(G, virtual_entry_node, original_roots):
    import networkx as nx

    target_list = [n for n in G.nodes if G.out_degree(n) == 0]
    full_node_order = list(nx.topological_sort(G))

    routes = []
    for t in target_list:
        for path in nx.all_simple_paths(G, source=virtual_entry_node, target=t):
            routes.append(path)

    all_nodes_in_routes = set()
    for r in routes:
        all_nodes_in_routes.update(r)
    V = list(all_nodes_in_routes)

    node_order = [n for n in full_node_order if n in V]

    setV = set(V)
    set_roots = set(original_roots)
    set_targets = set(target_list)
    as1 = list(setV - set_roots - set_targets - {virtual_entry_node})

    # as2 = routes
    return routes, V, as1, routes, target_list, node_order

In [430]:
def setup_game_parameters(V, routes, virtual_entry_node, target_list, as1):
    setV = set(V)
    set_targets = set(target_list)
    adv_list = list(setV - set_targets - {virtual_entry_node})
    n = len(as1)
    m = len(routes)
    L = len(adv_list)
    if n == 0:
        theta = [1.0 if L == 1 else (1.0 / L) for _ in range(L)]
    else:
        theta = [1.0 / n for _ in range(L)]
    return adv_list, theta, m

In [431]:
def build_game(G, virtual_entry_node, original_roots):
    """
    Builds the game elements:
      - routes
      - V
      - as1 (defender-check nodes)
      - as2 (the same as 'routes')
      - target_list
      - node_order
      - adv_list, theta, m (# of routes)
    Then prints debug statements similar to the R code.
    """
    routes, V, as1, as2, target_list, node_order = generate_game_elements(
        G, virtual_entry_node, original_roots
    )

    # Convert each path from list -> tuple (just for consistency)
    as2 = [tuple(path) for path in as2]

    adv_list, theta, m = setup_game_parameters(
        V=V,
        routes=as2,
        virtual_entry_node=virtual_entry_node,
        target_list=target_list,
        as1=as1
    )

    # Debug statements just like in R
    print("\n[DEBUG] node_order:", node_order)
    print("[DEBUG] target_list:", target_list)
    print("[DEBUG] routes:")
    for r in routes:
        print("   ", " -> ".join(map(str, r)))
    print("[DEBUG] V (unique nodes in routes):", V)
    print("[DEBUG] as1 (defender-check nodes):", as1)
    print("[DEBUG] adv_list (attacker start locs):", adv_list)
    print("[DEBUG] Theta:", theta)

    return routes, V, as1, as2, target_list, node_order, adv_list, theta, m

## Calculate PayOff Matrix

In [432]:
## Calculate PayOff Matrix
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    hardness = []
    for i in range(len(route) - 1):
        edge_data = graph[route[i]][route[i+1]]
        prob = float(edge_data.get('edge_probabilities', 1.0))
        hardness.append(prob)
    hardness = np.array(hardness, dtype=float)
    hardness = np.nan_to_num(hardness, nan=1.0)

    stop_probs = np.append(1 - hardness, 1.0)
    path_probs = np.concatenate(([1.0], np.cumprod(hardness)))
    pdfD = stop_probs * path_probs

    if pdfD.sum() > 0:
        pdfD /= pdfD.sum()
    return pdfD

In [433]:
def calculate_payoff_matrix_single_target(
    graph, as1, as2, V, adv_list, theta, random_steps_fn,
    attackRate, defenseRate, node_order, target_node
):
    """
    payoffs[i][j] = Probability the attacker *reaches* `target_node`.

    We'll insert debug statements that mimic the style used in R.
    We'll also reorder U by `node_order` and set zeros to 1e-7,
    then re-normalize, exactly like in the R code snippet.
    """
    payoffMatrix = []

    for defender_node in as1:

        row = []

        for path in as2:

            # Debug: show (defender_node, path, attackRate, defenseRate)
            print(f"\n[DEBUG] defender={defender_node}, "
                  f"path={path}, attackRate={attackRate}, defenseRate={defenseRate}")

            # Step 1: Build distribution U exactly like in R
            U = {v: 0.0 for v in V}

            # ========== Inner loop: for each adversary start location 'avatar' in advList =====
            for avatar in adv_list:
                L = {v: 0.0 for v in V}

                if avatar in path:
                    start_idx = path.index(avatar)
                    route = path[start_idx:]
                    
                    pdfD = random_steps_fn(route, attackRate, defenseRate, graph)

                    if defender_node in route:
                        cutPoint = min(route.index(defender_node), len(route) - 1)
                    else:
                        cutPoint = len(route) - 1

                    slice_sum = sum(pdfD[:cutPoint+1])
                    if slice_sum == 0.0:
                        payoffDistr = [0.0]*(cutPoint+1)
                        payoffDistr[cutPoint] = 1.0
                    else:
                        payoffDistr = [x / slice_sum for x in pdfD[:cutPoint+1]]

                    print(f"[DEBUG]   avatar={avatar}, route={route}, "
                          f"cutPoint={cutPoint}, payoffDistr={payoffDistr}")

                    for idx in range(cutPoint+1):
                        L[route[idx]] = payoffDistr[idx]

                else:
                    L[avatar] = 1.0

                # Weighted sum into U
                adv_index = adv_list.index(avatar)
                for v_ in V:
                    U[v_] += theta[adv_index] * L[v_]

            # Step 2: Normalize U
            total_U = sum(U.values())
            if total_U > 0.0:
                U = {v_: (U[v_] / total_U) for v_ in V}

            # Step 3: Reorder by node_order, then set zeros to 1e-7
            U_reordered = []
            for nd in node_order:
                val = U.get(nd, 0.0)
                if val < 1e-15:
                    val = 1e-7
                U_reordered.append(val)

            sumUr = sum(U_reordered)
            if sumUr > 0.0:
                U_reordered = [val / sumUr for val in U_reordered]

            debug_distr_str = ", ".join(
                [f"{node_order[i]}={U_reordered[i]:.6f}" for i in range(len(node_order))]
            )
            print(f"[DEBUG]   final distribution (reordered) U: {debug_distr_str}")

            if target_node not in node_order:
                p_target = 0.0
            else:
                t_idx = node_order.index(target_node)
                p_target = U_reordered[t_idx]

            print(f"[DEBUG]   => p_target={p_target:.6f}")

            row.append(p_target)

        payoffMatrix.append(row)

    return payoffMatrix

## Find Optimal Attacker / Defender Strategies

In [434]:
def solve_game(payoffs, as1, as2, logger=None):
    """
    Solve as a zero-sum game with standard linear programming.
    payoffs[i][j]: single numeric payoff for def-action i, att-action j.
    """

    if logger is None:
        import logging
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)

    payoffs = np.array(payoffs, dtype=float)
    num_def_actions = len(as1)
    num_att_actions = len(as2)

    # (1) Defender's LP: maximize v
    c_def = np.zeros(num_def_actions + 1)
    c_def[0] = -1.0  # because we minimize -v to maximize v

    A_ub = []
    b_ub = []
    for j in range(num_att_actions):
        row = np.zeros(num_def_actions + 1)
        row[0] = 1.0
        for i in range(num_def_actions):
            row[i+1] = -payoffs[i][j]
        A_ub.append(row)
        b_ub.append(0.0)

    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)

    A_eq = np.zeros((1, num_def_actions + 1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])

    bounds = [(None, None)] + [(0, None)] * num_def_actions

    res_def = linprog(c_def, A_ub, b_ub, A_eq, b_eq, bounds=bounds, method='highs')
    if not res_def.success:
        logger.info("Defender LP failed.")
        logger.info(res_def.message)
        return None

    x_star = res_def.x[1:]
    v_defender = res_def.x[0]

    # (2) Attacker's LP
    c_att = np.zeros(num_att_actions + 1)
    c_att[0] = -1.0

    A_ub_att = []
    b_ub_att = []
    for i in range(num_def_actions):
        row = np.zeros(num_att_actions + 1)
        row[0] = 1.0
        for j in range(num_att_actions):
            row[j+1] = -payoffs[i][j]
        A_ub_att.append(row)
        b_ub_att.append(0.0)

    A_ub_att = np.array(A_ub_att)
    b_ub_att = np.array(b_ub_att)

    A_eq_att = np.zeros((1, num_att_actions + 1))
    A_eq_att[0,1:] = 1.0
    b_eq_att = np.array([1.0])
    bounds_att = [(None, None)] + [(0, None)]*num_att_actions

    res_att = linprog(c_att, A_ub_att, b_ub_att, A_eq_att, b_eq_att,
                      bounds=bounds_att, method='highs')
    if not res_att.success:
        logger.info("Attacker LP failed.")
        logger.info(res_att.message)
        return None

    y_star = res_att.x[1:]
    v_attacker = res_att.x[0]

    return {
        'optimal_defense': dict(zip(as1, x_star)),
        'optimal_attack': dict(zip(as2, y_star)),
        'defender_value': v_defender,
        'attacker_value': v_attacker
    }

## Run The Game

In [435]:
def run_single_target_game(
    graph, as1, as2, V, adv_list, theta, node_order, random_steps_fn,
    attackRate_list,  # <-- Add or rename here
    defense_rate_list,
    target_node
):
    """
    For each (defenseRate, attackRate):
      - build payoff matrix = prob of attacker reaching target
      - convert to "defender payoff" = - prob
      - call solve_game
    """
    for defenseRate in defense_rate_list:
        for attackRate in attackRate_list:
            print(f"\n======================================")
            print(f"attackRate={attackRate}, defenseRate={defenseRate}")

            payoffs = calculate_payoff_matrix_single_target(
                graph, as1, as2, V, adv_list, theta,
                random_steps_fn, attackRate, defenseRate, node_order, target_node
            )
            # Defender payoff = - prob (attacker reaches target)
            payoffs_defender = [[-p for p in row] for row in payoffs]

            eq = solve_game(payoffs=payoffs_defender, as1=as1, as2=as2)
            if eq is None:
                print("No equilibrium found.")
                continue

            v_def = eq['defender_value']
            v_att = eq['attacker_value']
            print(f"Defender value = {v_def:.4f}")
            print(f"Attacker value = {v_att:.4f}")

            print("Optimal defense strategy (prob):")
            for d_node, prob in eq['optimal_defense'].items():
                print(f"   {d_node}: {prob:.4f}")

            print("Optimal attack strategy (prob):")
            for a_path, prob in eq['optimal_attack'].items():
                print(f"   Path {a_path}: {prob:.4f}")

            # If payoffs = -p_success, eq value ~ -p_success
            p_success_equilibrium = -v_def
            print(f"\nAttacker's success probability ~ {p_success_equilibrium:.4f}")


## Main method

In [436]:
def main():
    import logging
    from copy import deepcopy
    import os

    experiment_log_file = 'experiment_3.log'
    log_path = os.path.join(os.getcwd(), experiment_log_file)
    if os.path.exists(log_path):
        os.remove(log_path)

    logger = logging.getLogger()
    handler = logging.FileHandler(log_path, mode='w')
    handler.setFormatter(logging.Formatter('%(message)s'))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

    # 1) Load original MultiDiGraph
    work_graph = deepcopy(attack_graph)

    # 2) Process => single "virtual_target_node"
    processed_graph, original_roots, virtual_entry_node = process_graph(work_graph)

    # 3) Build game elements
    routes, V, as1, as2, target_list, node_order, adv_list, theta, m = build_game(
        processed_graph,
        virtual_entry_node=virtual_entry_node,
        original_roots=original_roots
    )

    # Must have at least one target
    if not target_list:
        print("No target_list found in graph. Exiting.")
        return

    single_target_node = target_list[0]

    # 4) Attack/defense rates
    attack_rate_list = [0]
    defense_rate_list = [0]

    # 5) Run single-target game
    run_single_target_game(
        graph=processed_graph,
        as1=as1,
        as2=as2,
        V=V,
        adv_list=adv_list,
        theta=theta,
        node_order=node_order,
        random_steps_fn=random_steps,
        attackRate_list=attack_rate_list,
        defense_rate_list=defense_rate_list,
        target_node=single_target_node
    )

    print(f"\nDone. See '{experiment_log_file}' for logs.")

if __name__ == "__main__":
    main()


[DEBUG] After process_target_node:
   Number of nodes in 'G_final' = 14
   Node labels: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 'attacker_entry_node', 'c("12","13","14","16")']

[DEBUG] node_order: ['attacker_entry_node', 1, 3, 4, 2, 5, 6, 7, 9, 11, 8, 10, 15, 'c("12","13","14","16")']
[DEBUG] target_list: ['c("12","13","14","16")']
[DEBUG] routes:
    attacker_entry_node -> 1 -> 5 -> 15 -> c("12","13","14","16")
    attacker_entry_node -> 3 -> 6 -> 8 -> 10 -> 15 -> c("12","13","14","16")
    attacker_entry_node -> 3 -> 6 -> 8 -> c("12","13","14","16")
    attacker_entry_node -> 3 -> 8 -> 10 -> 15 -> c("12","13","14","16")
    attacker_entry_node -> 3 -> 8 -> c("12","13","14","16")
    attacker_entry_node -> 4 -> 7 -> 10 -> 15 -> c("12","13","14","16")
    attacker_entry_node -> 4 -> 7 -> c("12","13","14","16")
    attacker_entry_node -> 2 -> 9 -> c("12","13","14","16")
    attacker_entry_node -> 2 -> 10 -> 15 -> c("12","13","14","16")
    attacker_entry_node -> 2 -> 11 -> c("12","13"

In [437]:
# def process_target_node(G):
#     """
#     Merges multiple sink nodes into a single 'virtual_target_node' with aggregated edges 
#     in a MultiDiGraph, then converts to DiGraph for simpler debugging.
#     """
#     targets = [n for n in G.nodes if G.out_degree(n) == 0]
#     if len(targets) > 1:
#         virtual_target = "virtual_target_node"
#         G.add_node(virtual_target)

#         # 1) For each old target t, replicate edges p->t as p->virtual_target_node
#         for t in targets:
#             preds = list(G.predecessors(t))
#             for p in preds:
#                 # G[p][t] is a dictionary of edge keys -> attribute dict
#                 for key, attr_dict in G[p][t].items():
#                     prob = attr_dict.get("edge_probabilities", 1.0)
#                     wt   = attr_dict.get("weight", -math.log(prob) if prob > 0 else float('inf'))
#                     G.add_edge(
#                         p, 
#                         virtual_target, 
#                         edge_probabilities=prob,
#                         weight=wt
#                     )

#         # Remove old target nodes
#         for t in targets:
#             G.remove_node(t)

#         # 2) Now unify multiple edges p->virtual_target_node with aggregator
#         aggregator_dict = {}
#         # We'll gather everything in aggregator_dict[(p, virtual_target)] = combined_probability
#         in_edges = list(G.in_edges(virtual_target, keys=True))  
#         # in_edges are (p, virtual_target, key)

#         for (p, vt, k) in in_edges:
#             attr_dict = G[p][vt][k]
#             p_i = attr_dict.get("edge_probabilities", 1.0)
#             if (p, vt) not in aggregator_dict:
#                 aggregator_dict[(p, vt)] = p_i
#             else:
#                 existing_p = aggregator_dict[(p, vt)]
#                 # union-of-probabilities aggregator
#                 new_p = 1 - (1 - existing_p)*(1 - p_i)
#                 aggregator_dict[(p, vt)] = new_p

#         # Remove all multi-edges to virtual_target
#         for (p, vt, k) in in_edges:
#             G.remove_edge(p, vt, key=k)

#         # Re-add a single edge for each aggregator result
#         for (p, vt), agg_prob in aggregator_dict.items():
#             if agg_prob <= 0:
#                 final_wt = float('inf')
#             elif agg_prob >= 1:
#                 final_wt = 0.0
#             else:
#                 final_wt = -math.log(agg_prob)

#             G.add_edge(
#                 p, 
#                 vt, 
#                 edge_probabilities=agg_prob,
#                 weight=final_wt
#             )

#     # Finally, convert from MultiDiGraph to DiGraph so debugging is straightforward
#     # (i.e. G[u][v] is a single dict with your 'edge_probabilities' and 'weight'.)
#     G_converted = nx.DiGraph()
#     G_converted.add_nodes_from(G.nodes(data=True))

#     for (u, v, k, data) in G.edges(keys=True, data=True):
#         # If we haven't yet added (u->v), just add it
#         if not G_converted.has_edge(u, v):
#             G_converted.add_edge(u, v, **data)
#         else:
#             # If there's still a possibility of parallel edges, we can decide:
#             # e.g., keep the existing one, or do a union-of-probabilities again.
#             # For simplicity, let's keep the existing edge.
#             pass

#     return G_converted
